In [2]:
from alpaca.data.live import StockDataStream
from datetime import datetime, timedelta
import pytz
import pandas as pd
import nest_asyncio
import asyncio
import os

# Applica patch per notebook o ambienti embedded (opzionale in PyCharm)
nest_asyncio.apply()

# Configura le API Alpaca
API_KEY = "AKNKP2ZFQPIEPD7XJBE2"
API_SECRET = "wGeVWSA3lAzp6zHwkdwcbU2w5n9EVC2OZj1B4YWe"

# Legge tickers S&P500
url = "https://raw.githubusercontent.com/datasets/s-and-p-500-companies/master/data/constituents.csv"
tic = pd.read_csv(url)
tickers = tic['Symbol'].tolist()
top_30_tickers = [
    "AAPL", "MSFT", "NVDA", "AMZN", "META", "BRK.B", "GOOGL", "AVGO", "TSLA", "GOOG",
    "LLY", "JPM", "V", "XOM", "NFLX", "COST", "UNH", "JNJ", "PG", "MA",
    "CVX", "MRK", "PEP", "ABBV", "ADBE", "WMT", "BAC", "HD", "KO", "TMO"
]
top_30_dict = {
    "AAPL": "Apple Inc.",
    "MSFT": "Microsoft Corp.",
    "NVDA": "NVIDIA Corp.",
    "AMZN": "Amazon.com Inc.",
    "META": "Meta Platforms Inc.",
    "BRK.B": "Berkshire Hathaway Inc.",
    "GOOGL": "Alphabet Inc. Class A",
    "AVGO": "Broadcom Inc.",
    "TSLA": "Tesla Inc.",
    "GOOG": "Alphabet Inc. Class C",
    "LLY": "Eli Lilly and Co.",
    "JPM": "JPMorgan Chase & Co.",
    "V": "Visa Inc.",
    "XOM": "Exxon Mobil Corp.",
    "NFLX": "Netflix Inc.",
    "COST": "Costco Wholesale Corp.",
    "UNH": "UnitedHealth Group Inc.",
    "JNJ": "Johnson & Johnson",
    "PG": "Procter & Gamble Co.",
    "MA": "Mastercard Inc.",
    "CVX": "Chevron Corp.",
    "MRK": "Merck & Co. Inc.",
    "PEP": "PepsiCo Inc.",
    "ABBV": "AbbVie Inc.",
    "ADBE": "Adobe Inc.",
    "WMT": "Walmart Inc.",
    "BAC": "Bank of America Corp.",
    "HD": "Home Depot Inc.",
    "KO": "Coca-Cola Co.",
    "TMO": "Thermo Fisher Scientific Inc."
}


# Dati raccolti
df_all_data = pd.DataFrame()

# Stream Alpaca
stream = StockDataStream(API_KEY, API_SECRET)

# Directory per salvataggi
os.makedirs("dati_stream", exist_ok=True)

# Fuso orario
italy_timezone = pytz.timezone("Europe/Rome")

# Ricezione trade
async def handle_trade_data(data):
    global df_all_data
    row = {
        "Ticker": data.symbol,
        "Timestamp": pd.to_datetime(data.timestamp, utc=True),
        "Price": data.price,
        "Size": data.size,
        "Exchange": data.exchange
    }
    df_all_data = pd.concat([df_all_data, pd.DataFrame([row])], ignore_index=True)
    print(df_all_data.tail(1))

# Salvataggio periodico
async def save_data_periodically(interval_sec=30):
    while True:
        await asyncio.sleep(interval_sec)
        if not df_all_data.empty:
            timestamp = datetime.now(italy_timezone).strftime("%Y%m%d_%H%M%S")
            path = f"dati_stream/stream_{timestamp}.csv"
            df_all_data.to_csv(path, index=False)
            print(f"💾 Salvato {len(df_all_data)} righe in {os.path.abspath(path)}")

# Funzione principale
async def main(duration_seconds=60):
    global df_all_data

    # Abbonati ai primi 30 ticker (Alpaca free limit)
    for symbol in top_30_tickers:
        stream.subscribe_trades(handle_trade_data, symbol)

    # Task salvataggio
    save_task = asyncio.create_task(save_data_periodically(interval_sec=20))

    # Stop dopo X secondi
    stop_time = datetime.now(italy_timezone) + timedelta(seconds=duration_seconds)

    # Avvia streaming
    stream_task = asyncio.create_task(stream.run())

    # Attesa
    while datetime.now(italy_timezone) < stop_time:
        await asyncio.sleep(1)

    # Stop streaming
    await stream.stop()

    # Ferma salvataggio periodico
    save_task.cancel()

    # Salvataggio finale
    if not df_all_data.empty:
        final_timestamp = datetime.now(italy_timezone).strftime("%Y%m%d_%H%M%S")
        final_path = f"dati_stream/stream_{final_timestamp}_FINAL.csv"
        df_all_data.to_csv(final_path, index=False)
        print(f"✅ Salvataggio finale con {len(df_all_data)} righe in {os.path.abspath(final_path)}")

    print("✅ Raccolta dati terminata.")

# Avvio dello script (solo in esecuzione diretta)
if __name__ == "__main__":
    asyncio.run(main(duration_seconds=60))


error during websocket communication: auth failed
Traceback (most recent call last):
  File "c:\Users\miklo\Desktop\Big data Finance\big-data-finance\Lib\site-packages\alpaca\data\live\websocket.py", line 343, in _run_forever
    await self._start_ws()
  File "c:\Users\miklo\Desktop\Big data Finance\big-data-finance\Lib\site-packages\alpaca\data\live\websocket.py", line 135, in _start_ws
    await self._auth()
  File "c:\Users\miklo\Desktop\Big data Finance\big-data-finance\Lib\site-packages\alpaca\data\live\websocket.py", line 126, in _auth
    raise ValueError(msg[0].get("msg", "auth failed"))
ValueError: auth failed
error during websocket communication: auth failed
Traceback (most recent call last):
  File "c:\Users\miklo\Desktop\Big data Finance\big-data-finance\Lib\site-packages\alpaca\data\live\websocket.py", line 343, in _run_forever
    await self._start_ws()
  File "c:\Users\miklo\Desktop\Big data Finance\big-data-finance\Lib\site-packages\alpaca\data\live\websocket.py", line 

keyboard interrupt, bye


TimeoutError: 